# Custom loss implementations - Huber loss

- Implementation using functions
- Hyperparameter implementation using wrapper function
- Hyperparameter implementation using class

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

## Huber Loss implementation

### Dataset

In [2]:
# inputs
xs = np.array([-1.0,  0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)

# labels
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

### Function and wrapper function implementation

In [3]:
# Loss function
threshold = 1.2

def my_huber_loss(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) <= threshold
    small_error_loss = tf.square(error) / 2
    big_error_loss = threshold * (tf.abs(error) - (0.5 * threshold))
    return tf.where(is_small_error, small_error_loss, big_error_loss)

In [4]:
def my_huber_loss_with_threshold(threshold):
    def my_huber_loss(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) <= threshold
        small_error_loss = tf.square(error) / 2
        big_error_loss = threshold * (tf.abs(error) - (0.5 * threshold))
        return tf.where(is_small_error, small_error_loss, big_error_loss)

    # return the inner function
    return my_huber_loss

In [5]:
model = tf.keras.Sequential([keras.layers.Dense(units=1, input_shape=[1])])
model.compile(optimizer='sgd', loss=my_huber_loss_with_threshold(threshold=1.2))
model.fit(xs, ys, epochs=500, verbose=0)

/Users/opmule/miniforge3/envs/ai_tensorflow/lib/python3.10/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2026-01-12 21:16:32.424827: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2026-01-12 21:16:32.424858: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2026-01-12 21:16:32.424862: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.92 GB
I0000 00:00:1768232792.424877 1711222 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1768232792.424899 1711222 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:

In [6]:
print(model.predict(np.array([[10.0]])))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
[[18.805712]]


### Implement loss with Class

In [7]:
xs = np.array([-1.0, 0.0, 1.0, 2.0, 3.0, 4.0], dtype=float).reshape(-1, 1)
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float).reshape(-1, 1)

In [8]:
from tensorflow.keras.losses import Loss

class MyHuberLoss(Loss):

    # initialize instance attributes
    def __init__(self, threshold=1):
        super().__init__()
        self.threshold = threshold

    # compute loss
    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) <= self.threshold
        small_error_loss = tf.square(error) / 2
        big_error_loss = self.threshold * (tf.abs(error) - (0.5 * self.threshold))
        return tf.where(is_small_error, small_error_loss, big_error_loss)

In [11]:
model_class = tf.keras.Sequential([keras.layers.Dense(units=1, input_shape=[1])])
model_class.compile(optimizer='sgd', loss=MyHuberLoss(threshold=1.02))
model_class.fit(xs, ys, epochs=500, verbose=0)

In [12]:
print(model_class.predict(np.array([[10.0]])))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
[[18.581604]]
